###Check Python version exist in your system and if you don't have python use command below.

In [ ]:
!apt-get install python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python is already the newest version (2.7.15~rc1-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
!python --version

Python 3.7.12


###Install data parsing library bs4 to get import BeautifulSoup according to your python version.

In [ ]:
!pip install bs4

###Install Tweepy to get access of Twitter API.

In [ ]:
!pip install tweepy 

###Install lxml to allows easy handling of XML file, and can also be used for web scraping.

In [ ]:
!pip install lxml

###Install transformers use for hugging face to perform sentiment analysis.

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 11.7 MB/s 
     |████████████████████████████████| 6.5 MB 30.6 MB/s 
     |████████████████████████████████| 895 kB 40.8 MB/s 


###Import all the libraries which we install.

In [ ]:
import pandas as pd
import ssl
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import tweepy
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


### Create a function to get access of twitter api and tweets fetching.
#### How to get twitter API
i) You have to open url 'https://developer.twitter.com/' \
ii) SignUp using your existing twitter account, if you don't have twitter account firstly create it using 'https://twitter.com/' \
iii) Go to dashboard and create app \
iv) Fill required details, make sure you are email and contact number verified. \
v) There you can get your api's \
vi) Where you have to copy (API Key and Secret) from Consumer Keys and (Access Token and Secret) from Authentication Tokens. \
vii) Make sure it has to elevated access to get the data from twitter.

In [ ]:
def twitter_news(keyword,columns):
    twd=[]
    # Access Twitter API 
    ak = ['Paste your api_key here']
    aks = ['Paste your api_key_secret here']
    at = ['Paste your access_key here']
    ats = ['Paste your access_key_secret here']
    # Authentication Twitter API
    auth = tweepy.OAuthHandler(ak, aks)
    auth.set_access_token(at, ats)
    api = tweepy.API(auth)
    tweets_parse = tweepy.Cursor(api.search, q=keyword, count=100, tweet_mode='extended').items(6000)
    for tweet in tweets_parse:
        twd.append([tweet.created_at,tweet.full_text,tweet.user.screen_name])
    twitter_df = pd.DataFrame(twd, columns=columns)
    return twitter_df

### Create a function for google news fetching using bs4(soup).

In [ ]:
def google_news(xml_news_url,columns):
    google_data=[]
    context = ssl._create_unverified_context()
    Client = urlopen(xml_news_url, context=context)
    xml_page = Client.read()
    Client.close()
    soup_page = soup(xml_page, "xml")
    news_list = soup_page.findAll("item")
    # Parse February data
    for news in news_list:
        if "Feb" in news.pubDate.text:
            google_data.append([news.pubDate.text,news.title.text,news.link.text])
    google_df=pd.DataFrame(google_data,columns=columns)
    return google_df

### Create function for sentiment analysis.

In [ ]:
def sentiment_analysis(df):
  l,l1=[],[]
  for i in df['Tweet content/news headline']:
    l+=[sentiment_pipeline(i)[0]['label']] 
    l1+=[sentiment_pipeline(i)[0]['score']] 
  df['label'] = l
  df['score'] = l1
  return df

### Call Both the function(twitter_news and google_news) and concatinate it and perform sentiment analysis.
---



In [ ]:
columns = ['Date of News/Twitter','Tweet content/news headline','source of news / person name who has tweeted']
twitter_keyword = 'Green Hydrogen'
google_url = 'https://news.google.com/rss/search?q=Green+Hydrogen+when:35d&hl=en-IN&gl=IN&ceid=IN:en'
for_twi_news = twitter_news(twitter_keyword,columns)
for_google_news = google_news(google_url,columns)
result = [for_twi_news,for_google_news]
df = pd.concat(result,ignore_index=True)
overall_data = sentiment_analysis(df)
overall_data

In [ ]:
overall_data.to_csv('twitter_google_data.csv',index=False)